*Note: this notebook is the continuation of the [previous notebook](), intended to be show in slideshow mode.

# Data Preparation 2

In today's second lecture we'll cover:
1. Numerical transformations
2. Changes in granularity: aggregation and hierarchies

In [ ]:
## we'll use the Lahman baseball database in our initial examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://jmh@127.0.0.1:5432/baseball
        
import trifacta as tf
import pandas as pd

# 1. Numerical Transformations
- Super common requirement. 
- Calculations look fairly similar across languages
  - We'll use SQL today
- There are various types of calculations to consider:
  - Scalar functions
  - Aggregate functions
  - Window functions: basic, inverse-distribution, hypothetical-set

## Scalar functions
- Recall *scalar*: a tensor of dimension $0$
  - i.e. a value in some *field*. 
<img src="files/tensors.png">
- Colloquially: a scalar value in any *atomic* data type
- Scalar function: function on scalar values
    - constants and attributes on individual cells
- Relational algebra: subscripts of $\pi, \sigma$ and $\Join$
    - e.g. $\pi_{a, f(b)}(R)$. 
- Dataflow (Spark or Pandas): applied by the `map` function.

Identify all the functions in the next query. Are they scalar?

In [ ]:
%%sql
WITH yearnum AS
  (SELECT yearid, (yearid % 100) as year
     FROM batting
  )
SELECT yearid, CONCAT('''', LPAD(year::text, 2, '0')) as year
  FROM yearnum
 LIMIT 5;

#### Scalar functions are typically quick!
- computed individually on each record in your data
- system can run them in parallel on many different records
- system can release memory after each call

#### Scalar functions in a DBMS
- Executed "on-the-fly" as tuples are emitted
- E.g. after accessing the data (`Seq Scan`, `Index Scan`)
- Consider this query (a simple "flattening" of the above):

In [ ]:
%%sql
EXPLAIN (VERBOSE true)
SELECT yearid, CONCAT('''', LPAD((yearid % 100)::text, 2, '0')) as year
  FROM batting;

#### Scalar functions mentioning multiple tables?
- Happens right as those tables are joined up.

In [ ]:
%%sql
EXPLAIN (VERBOSE true)
SELECT p.playerid, p.so - b.so
  FROM pitching p INNER JOIN batting b ON p.playerid=b.playerid;

#### User-Defined Functions (UDFs)
- Supported in some DBMSs, Spark, etc.
- Define scalar functions of your own
- Use them as you would use built-in functions.
- Postgres supports many languages including Python

In [ ]:
%%sql
CREATE LANGUAGE plpythonu;

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION pyhash(s text)
  RETURNS text
AS $$
  ## Python text goes here, can reference variable s
  import hashlib
  m = hashlib.sha256()
  m.update(s)          # here's s!
  return m.hexdigest() # return a text
$$ LANGUAGE plpythonu;

In [ ]:
%%sql
SELECT pyhash('Joe'), pyhash('Joel');

## Aggregate Functions
Aggregate functions: input is a *set* or *vector* of values
- Linear alg: $\sum_i$ and $\prod_i$
- More general languages: a family of aggregate functions built in.
    - Usually can define your own. "UDA"s, Reduce.

#### Typical Built-in Aggs
[Examples from PostgreSQL](https://www.postgresql.org/docs/current/functions-aggregate.html) include:
   - Univariate functions on sets of numbers: 
     - `min`, `max`, `sum`, `avg`, `min`, `max`, `stddev`, `variance`, etc.
   - Bivariate functions on sets of numbers:
     - correlation (e.g. `corr`), covariance (e.g. `covar_samp`), regression (e.g. `regr_sxx`), etc
   - Univariate functions on ordered lists of numbers:
     - `percentile_cont, percentile_disc`

Also univariate functions on ordered lists of strings
   - `string_agg`, `array_agg`, `json_agg`

Let's look at some annual homerun (`HR`) statistics.

In [ ]:
%%sql
SELECT namefirst, namelast, min(hr), max(hr), avg(hr), stddev(hr), sum(hr)
  FROM batting b, master m
 WHERE b.playerid = m.playerid
 GROUP BY namelast, namefirst
 HAVING stddev(hr) IS NOT NULL
 ORDER BY max desc
 LIMIT 10;

In [ ]:
%%sql 
SELECT namelast, json_agg(namefirst)
  FROM master
 WHERE namelast IN ('Ruth', 'Aaron', 'DiMaggio', 'Rodriguez')
GROUP BY namelast
ORDER BY count(*) DESC;

## Window Functions
- Recall: 1 value in output for each "window" of input values
- Any aggregate function can be used in a window!
- Order-based aggregates (thanks to ordered windows).

Let's do 3-year windowed statistics on homeruns for Bonds and Ruth:

In [ ]:
%%sql
SELECT namefirst, namelast, yearid, HR,
       rank() OVER (ORDER BY HR DESC),
       avg(HR)    OVER (PARTITION BY b.playerid ORDER BY yearid ROWS 3 PRECEDING) as avg_3yr,
       lag(HR, 7) OVER (PARTITION BY b.playerid ORDER BY yearid) as previous,
       lag(HR, 2) OVER (PARTITION BY b.playerid ORDER BY yearid) as lag2
  FROM batting b, master m
 WHERE m.playerid = b.playerid
   AND (namelast = 'Bonds' or namelast = 'Ruth')
 ORDER BY HR DESC
 LIMIT 10;

### Inverse Distribution Window Functions
- The value at a particular "position" in a distribution
- SQL: use `WITHIN GROUP (ORDER BY...)`.

Example: "Tukey numbers" (min, quartiles, max, avg) for home runs plus "p99"
   - since homeruns/year is heavy-tailed distribution

In [ ]:
%%sql
SELECT MIN(HR),
       percentile_cont(0.25) WITHIN GROUP (ORDER BY HR) AS p25,
       percentile_cont(0.50) WITHIN GROUP (ORDER BY HR) AS median,
       percentile_cont(0.75) WITHIN GROUP (ORDER BY HR) AS p75,
       percentile_cont(0.99) WITHIN GROUP (ORDER BY HR) AS p99,
       MAX(HR),
       AVG(HR)
  FROM batting
 LIMIT 10;

### Hypothetical-Set Window Functions
- the position of a value in a distribution *even if the value wasn't in the data*
- PostgreSQL: `rank`, `dense_rank`, `percent_rank` and `cume_dist`
- Would I be in the top 0.1% if I hit `hrs` homeruns?

In [ ]:
hrs = 4

In [ ]:
%%sql
SELECT :hrs as hypothetical,
       rank(:hrs) WITHIN GROUP (ORDER BY HR DESC),
       dense_rank(:hrs) WITHIN GROUP (ORDER BY HR DESC),
       percent_rank(:hrs) WITHIN GROUP (ORDER BY HR DESC) * 100 AS pct_rank,
       cume_dist(:hrs) WITHIN GROUP (ORDER BY HR)
  FROM batting
 LIMIT 10;

# 2. Changes in Granularity

## Changes in Granularity: Overview
Super common. Data is often recorded/released at different granularities.

**Units**: Numerical data is *measured*! Usually in hierarchy of units: 
- sec->hours->minutes->days
- mm->cm->m->km

Sometimes need to translate across hierarchies
- in->ft->mi

**Discretization**: Granularity is inherent in the notion of measurement. 
- *the temperature is `68` degrees Fahrenheit.* 
- "Rounded" (*discretized*) from a continuous value in the physical world
- Encoded as a fixed number of bits. 
- Result of both sensing devices and digitization.

## Explicit Hierarchies
Many hierarchical *models* of the world.

- Domain, Kingdom, Phylum, Class, Order, Family, Genus, Species. 
- City, County, State, Nation
- {Cars, Trucks, Planes} -> Vehicles

One standard encoding: `IsA` pairs
- basically a child/parent pairs in a tree:

| child | parent |
:-------|:----------|
| truck | vehicle |
| car   | vehicle |
| airplane | vehicle |
| Gulfstream 650 | airplane|

Also [semantic triples](https://en.wikipedia.org/wiki/Semantic_triple) a la [RDF]:

| subject | predicate | object |
| :------ | :-------- | :----- |
| airplane  | IsA      | vehicle |
| Gulfstream 650 | IsA | airplane |
| Joe | ReportsTo | John |
| John | ReportsTo | Tsu Jae |
| Tsu Jae | ReportsTo | Carol |
| Alvin | Teaches | Joe |
| Joe | Teaches | Alvin |

## A Note on Time (both kinds!)
Data is rarely true across all of time.
- Data should contain temporal attributes!

There are typically two relevant times for a datum:
- *Transaction Time*: the time a datum is recorded
- *Valid Time*: the time range when the datum is considered to be true

This is sometimes referred to as a *bitemporal* model for data.

*All data* is bitemporal, not just physical phenomena. 
- "Enrique is taking Data Engineering 101"
- "Samy bought a goldfish on eBay"

- "Transaction time" a bit circumstantial
- "Valid time" is very application-specific

Hierarchies are common (msec < sec < min < hour < day < month < year < ...)
- complicated by issues like timezones!

## A Note on Space
All physical phenomena also have a geolocation. 

- Can be encoded in many ways
- (latitude, longitude)
- place-name
- postal code

Explicit and often complex hierarchies of place-names/codes
- often not strict hierarchies, have overlaps
- because of politics!
- e.g. congressional districts may span parts of counties

## Everything Exists in Space-Time
- The four most common dimensions in data
- Great unboxing tricks: maps and [rug plots](https://en.wikipedia.org/wiki/Rug_plot)!
- Inherently multi-granularity
    - Time subdivisions are periodic! (seasons, months, etc)

You may need a special system!
- GIS: Geographic Information Systems (e.g. [PostGIS](https://postgis.net))
- Temporal Database: (e.g. [Timescale])(https://timescale.com)


Generally: general-purpose databases and tools.

But always keep space and time in mind.

## Granularity Transformations

In [ ]:
## we'll use the GNIS data in the following examples.
## The file: https://geonames.usgs.gov/docs/stategaz/NationalFile.zip
## The site: https://www.usgs.gov/core-science-systems/ngp/board-on-geographic-names/download-gnis-data
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://jmh@127.0.0.1:5432/gnis

In [ ]:
%sql \d national

In [ ]:
%sql SELECT * FROM national LIMIT 2;

### Quantization: Numerical Granularity
Simplest form: the number of "significant digits"
- Typically manipulate via arithmetic
- Or powers-of-10 via string prefixes:

In [ ]:
%%sql
SELECT elevation_meters, 
       (elevation_meters / 100) :: integer AS quantized,
       ((elevation_meters / 100) :: integer) * 100 AS round_to_100,
       substring(elevation_meters :: text, 1, 2),
       concat(substring(elevation_meters :: text, 1, 2), '00') AS substring2
  FROM national
 LIMIT 5;

The above is powers of 10.
- Can also quantize $n$-bit numbers to fewer bits. 
    - Common in Deep Learning pipelines. 

Each bit position is a power of two. 
- Shifting $b$ bits right (`>> b`) divides by $2^b$, drops bits. 
- Shifting $b$ bits left (`<< n`) multiplies by $2^b$. 
- Shifting right-then-left by $b$
    - Keep only the leading $n-b$ bits
    - Pad the right with 0

In [ ]:
%%sql
SELECT elevation_meters, 
       (elevation_meters::integer::bit(12)) AS bit12, 
       (elevation_meters::integer::bit(12) >> 3) AS rightshifted, 
       ((elevation_meters::integer::bit(12) >> 3) << 3) AS rightleftshifted,
       ((elevation_meters::integer::bit(12) >> 3) << 3)::integer AS round_down_8
  FROM national
 LIMIT 5;


   - right-then-left shift rounds down to the nearest multiple of $2^b$!

## Quantization and Domain Size
Quantizing to fewer bits means fewer distinct values
- akin to assigning "bin" numbers 

Take our 12-bit numbers and quantize down to the leading 4 bits.
- How many distinct values do you expect?

In [ ]:
%%sql
EXPLAIN (verbose)
WITH shifts AS (
SELECT elevation_meters, 
       (elevation_meters::integer::bit(12)) AS bit12, 
       (elevation_meters::integer::bit(12) >> 8) AS rightshifted, 
       ((elevation_meters::integer::bit(12) >> 8) << 8)::integer AS round_to_256,
       ((elevation_meters::integer::bit(12) >> 8) << 8)::integer % 256 AS test
  FROM national
)
SELECT COUNT(DISTINCT elevation_meters) AS elevation_meters_cnt,
       COUNT(DISTINCT bit12) AS bit12_cnt,
       COUNT(DISTINCT rightshifted) AS rightshift_cnt,
       COUNT(DISTINCT round_to_256) AS rounded_cnt
  FROM shifts;

### Rollup/Drill-Down Aggregation
"Roll up" = transform to be *coarser* grain.
"Drill down" = transform to be *finer* grain

Essentially GROUP BY/Aggregation with different groups.
- In the R community this pattern is called  *Split-Apply-Combine*.


Let's start with county-level data on elevations

In [ ]:
%%sql
SELECT state_numeric, county_numeric, avg(elevation_meters), stddev(elevation_meters), count(*)
  FROM national TABLESAMPLE BERNOULLI(10)
 GROUP BY state_numeric, county_numeric;

Now let's roll up to state level. 
- Watch the GROUP BY clause. 
- More or fewer columns? 
- Can we choose any columns we want?

In [ ]:
%%sql
CREATE OR REPLACE VIEW state_elevations AS
SELECT state_numeric, avg(elevation_meters) AS avg, stddev(elevation_meters) AS stddev, count(*) AS cnt
  FROM national TABLESAMPLE Bernoulli(10)
 GROUP BY state_numeric;

SELECT * FROM state_elevations;

Now let's drill down from `state_numeric` into `feature_class` as well:

In [ ]:
%%sql
SELECT state_numeric, feature_class, avg(elevation_meters) AS avg, stddev(elevation_meters) AS stddev, count(*) AS cnt
  FROM national TABLESAMPLE Bernoulli(10)
 GROUP BY state_numeric, feature_class
 ORDER BY count(*) DESC;

And roll up again to just `feature_class`!

In [ ]:
%%sql
SELECT feature_class, avg(elevation_meters) AS avg, stddev(elevation_meters) AS stddev, count(*) AS cnt
  FROM national TABLESAMPLE Bernoulli(10)
 GROUP BY feature_class
 ORDER BY count(*) DESC
 LIMIT 10;

And drill down into `state_numeric` and `county_numeric`!

In [ ]:
%%sql
SELECT state_numeric, county_numeric, feature_class, count(*), avg(elevation_meters)
  FROM national TABLESAMPLE Bernoulli(10)
 GROUP BY state_numeric, county_numeric, feature_class
 ORDER BY count(*) DESC
LIMIT 10;

- Is there a key in the schema of the output? Why (not)?
- What would this mean for PIVOTing?

### Rollup/Drilldown & Tensors
Consider a GROUP BY query with 3 grouping columns and 1 aggregate column:
```
SELECT state_numeric, county_numeric, 
       feature_class, count(*)
  FROM national
 GROUP BY state_numeric, county_numeric, feature_class;
```
*This returns a (sparse) 3-dimensional tensor!*

### Connection to Statistics
- GROUP BY / `count(*)` queries are *empirical probability distributions!*
  - (Well, if you normalize by the overall count.)
  - (Do it in SQL!)

- Roll-up (fewer grouping columns) on these queries is *marginalization* 
  - I.e. computing the [*marginal* distribution](https://en.wikipedia.org/wiki/Marginal_distribution) 
  - Sums the probabilities across a subset of dimensions
- Roll-up is the natural generalization to other aggregation functions
- We will revisit this when we talk about Data Cubes.

### Drill-Down via Modeling
What if we don't actually have the drill-down data
- Maybe we have a statistical model of it! 
- We can sample from the model. 
- Be clear with your "users" that this is simulated data!

E.g. a Gaussian (Normal) distribution:

In [ ]:
%%sql
SELECT * from normal_rand(10, 0, 1);

We can join against this to generate normally-distributed elevations!
- This is a table-valued function! 
- We "call" a table-valued function with a [LATERAL JOIN](https://www.postgresql.org/docs/current/queries-table-expressions.html)
- Essentially forces a join order

In [ ]:
%%sql
WITH state_cty AS
(SELECT s.state_numeric, f.fips as county_numeric, s.avg, s.stddev, s.cnt
  FROM state_elevations s, fips_counties f
  WHERE s.state_numeric = f.state_numeric
)
SELECT s.*, n.n AS elevation_meters, true as elevation_meters_simulated
  FROM state_cty s
       LATERAL normal_rand(s.cnt::integer, s.avg, s.stddev) AS n
LIMIT 10;

## Assembling and Joining to an Explicit Hierarchy
Let's return to the Baseball database. In baseball we have an explicit hierarchy:
- Players play for Teams
- Teams are in Divisions
- Divisions are in Leagues
If we want to do rollups and drilldowns on this hiearchy, we need data that looks like:
```
(playerid, teamid, divisionid, leagueid, yearid, thing_we_want_to_aggregate)
```
We will have to transform our data to get it into this structure!

In [ ]:
## we'll use the Lahman baseball database in our initial examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://jmh@127.0.0.1:5432/baseball

In [ ]:
%%sql
select * from appearances where yearid > 1970 limit 2;

In [ ]:
%%sql
select * from teams limit 1;

In [ ]:
%%sql
select a.playerid, a.teamid, t.divid, a.*
  from appearances a NATURAL JOIN teams t
 where a.yearid = 2015
 limit 100;

This hierarchy is an example of views we can use!

In [ ]:
%%sql
CREATE OR REPLACE VIEW bball_tree AS
SELECT DISTINCT a.playerid, a.teamid, t.divid, a.lgid, a.yearid
  FROM appearances a NATURAL JOIN teams t;

Now if we want to rollup and drill-down home run statistics on this hierarchy, we can do it! Remember our old homerun query?

In [ ]:
%%sql
SELECT namefirst, namelast, min(hr), max(hr), avg(hr), stddev(hr), sum(hr)
  FROM batting b, master m
 WHERE b.playerid = m.playerid
 GROUP BY namelast, namefirst
 ORDER BY max desc
 LIMIT 10;

Let's set it up for rollup/drilldown on the bball_tree hierarchy! To do this, we join each raw master (person) record with the associated bball_tree by `(playerid, yearid)` in a CTE, and use the result for rollup/drilldown.

In [ ]:
%%sql
WITH batting_tree AS
(
    SELECT b.*, t.divid
    FROM batting b, bball_tree as t
    WHERE b.playerid = t.playerid
      AND b.yearid = t.yearid 
)
SELECT b.playerid, b.teamid, b.lgid, b.divid, b.yearid, namelast, namefirst,
       namefirst, namelast, min(hr), max(hr), avg(hr), stddev(hr), sum(hr)
FROM batting_tree b, master m
WHERE b.playerid = m.playerid
GROUP BY b.playerid, b.teamid, b.lgid, b.divid, b.yearid, namelast, namefirst
ORDER BY max(hr) DESC
limit 10;


### Walking a Hierarchy of Arbitrary Depth
Let's look at a table with a complex hierarchy: the taxonomic hierarchy of biology from US [National Center for Biotechnology Information](https://www.ncbi.nlm.nih.gov/).

In [ ]:
## we'll use the NCBI taxonomy data in remaining examples.
## The file: https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/new_taxdump/new_taxdump.tar.Z
## The site: https://www.ncbi.nlm.nih.gov/books/NBK53758/
## Munge the nodes.dmp file with:
## cat nodes.dmp | tr '\t'  , | sed s/\,\|\,/\,/g | sed s/\|//g  > nodes_clean.dmp
## load with %copy nodes from 'nodes_clean.dmp' with csv delimiter ',';
## %alter table nodes add primary key (tax_id);
## Munge the names.dmp file with:
## cat names.dmp | tr '\t'  '|' | sed s/\|\|\|/\"\|\"/g | sed s/^/\"/ | sed s/\|\|$/\"/ > names_clean.dmp
## load with %copy names from '/Users/jmh/Downloads/new_taxdump/names_clean.dmp' with csv delimiter '|';
## %create index tax_id_ix on names(tax_id);
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://jmh@127.0.0.1:5432/taxdump

In [ ]:
%%sql
SELECT *
  FROM nodes
 ORDER BY tax_id
 LIMIT 5;

In [ ]:
%%sql
SELECT *
  FROM names
 ORDER BY tax_id
 LIMIT 5;

In [ ]:
%%sql
SELECT name_txt, nodes.* 
  FROM nodes NATURAL JOIN names
 WHERE name_txt = 'Animalia';

To Drill down below *Animalia*, we can use a *self-join* on nodes!

In [ ]:
%%sql
SELECT parent.tax_id, parent.rank, child.tax_id, child.rank
  FROM nodes parent, nodes child
 WHERE child.parent = parent.tax_id
   AND parent.tax_id = 6073;

Now let's get the names of these nodes!

In [ ]:
%%sql
WITH edges AS (
    SELECT parent.tax_id AS pid, parent.rank as prank,
           child.tax_id AS cid, child.rank as crank
      FROM nodes parent, nodes child
     WHERE child.parent = parent.tax_id
)
SELECT pid, prank, parent.name_txt as ptext, cid, crank, child.name_txt as ctext
  FROM names parent, names child, edges
 WHERE parent.tax_id = edges.pid
   AND child.tax_id = edges.cid
   AND parent.tax_id = 33208
 LIMIT 10;

Argh! Synonyms! There are many names for the same `tax_id`! Let's aggregate them up into unique `(pid, cid)` rows.

In [ ]:
%%sql
WITH edges AS (
    SELECT parent.tax_id AS pid, parent.rank as prank,
           child.tax_id AS cid, child.rank as crank
      FROM nodes parent, nodes child
     WHERE child.parent = parent.tax_id
)
SELECT pid, edges.prank, json_agg(parent.name_txt), cid, edges.crank, json_agg(child.name_txt)
  FROM names parent, names child, edges
 WHERE parent.tax_id = edges.pid
   AND child.tax_id = edges.cid
   AND parent.tax_id = 33208
 GROUP BY pid, edges.prank, cid, edges.crank;

Now we can drill down 2 levels of the hierarchy. What about three levels?

In [ ]:
%%sql
WITH double_edges AS (
    SELECT grandparent.tax_id AS gid, grandparent.rank as grank, 
           parent.tax_id AS pid, parent.rank as prank,
           child.tax_id AS cid, child.rank as crank
      FROM nodes grandparent, nodes parent, nodes child
     WHERE child.parent = parent.tax_id
       AND parent.parent = grandparent.tax_id
)
SELECT gid, grank, json_agg(grandparent.name_txt), 
       pid, prank, json_agg(parent.name_txt), 
       cid, crank, json_agg(child.name_txt)
  FROM names grandparent, names parent, names child, double_edges de
 WHERE parent.tax_id = de.pid
   AND child.tax_id = de.cid
   AND grandparent.tax_id = de.gid
   AND grandparent.tax_id = 33208
 GROUP BY gid, grank, pid, prank, cid, crank;

You can keep going deeper! 
- But it gets tedious in SQL. Why?

# Summing Up!

#### Numerical Functions
Know when to use what kind of function! And how they are interleaved with relational constructs like project, select, GROUP BY, join, etc.

- Scalar functions
  - built-in
  - [UDFs](https://www.postgresql.org/docs/current/xfunc.html)
- Aggregate functions
  - [built-in](https://www.postgresql.org/docs/current/functions-aggregate.html)
  - [UDAs](https://www.postgresql.org/docs/current/xaggr.html)
- [Window functions]()
  - basic aggs and order-based aggs
  - inverse-distribution
  - hypothetical-set
 - [Postgres Docs](https://www.postgresql.org/docs/current/functions.html)

#### Granularity Transformation
Understand why data comes in different granularities, and how to transform data to a desired granularity.
- Features of spatial data and political boundaries
- Bitemporal data
- Quantization: numerical transformation by powers of 10, 2, etc.
  - reduces the number of distinct values correspondingly
- Rollup/Drilldown constructs
  - more GROUP BY columns = drilldown
  - fewer GROUP BY columns = rollup
- Relational / Tensor connections
  - GROUP BY creates a tensor
  - Rollup is akin to Marginalization
- Explicit Hierarchies
  - Create a hierarchy table you can rollup/drilldown
  - Join your data to that table!